In [1]:
import ctypes
import json

dll = ctypes.WinDLL(f'./Ft8ChartGen.dll')
dll.SquareToItuZone.argtypes = [ctypes.c_wchar_p]

info_path = r"./stationInfo-yearto-2021-10-01.sql"
zones_path = r"./zones.json"
report_path = r"./report-2021-09-24.sql"
spots_path =  r"./spots.csv"

In [2]:
%%time

pref = 'INSERT  IGNORE INTO `stationInfo` VALUES ('
zones = {}

with open(info_path, mode='r', encoding='UTF-8') as file:
    for line in file:
        if line.startswith(pref): 
            line = line[len(pref):-3]
            for record in line.split('),('):
                fields = record.split(',')
                if fields[4] != 'NULL':
                    square = fields[4][1:7].upper().replace('\'', '')
                    zone = dll.SquareToItuZone(square)
                    zones[fields[0]] = zone

with open(zones_path, 'w') as f:
    f.write(json.dumps(zones))

Wall time: 16.6 s


In [3]:
%%time

with open(zones_path, 'r') as f:
    zones =  json.loads(f.read())
    
print(len(zones))

1005938
Wall time: 546 ms


In [4]:
%%time

pref = 'INSERT INTO `report` VALUES ('
spot_count = 0

with open(report_path, mode='r', encoding='UTF-8') as in_file:
    with open(spots_path, 'w') as out_file:
        for line in in_file:
            if line.startswith(pref):
                line = line[len(pref):-3]
                for record in line.split('),('):
                    fields = record.split(',')
                    if fields[4] == "'FT8'":
                        zone1 = zones.get(f'{fields[1]}', 0)
                        zone2 = zones.get(f'{fields[2]}', 0)
                        spot = f'{fields[0]},{fields[9]},{fields[14][1:-1]},{min(zone1, zone2)},{max(zone1, zone2)}'
                        out_file.write(f'{spot}\n')
                        spot_count += 1
print(spot_count)                        

53154249
Wall time: 2min 59s
